# CS 61A Fall 2025 Notes

```sh
wget https://cs61a.org/lab/lab00/lab00.zip
unzip lab00.zip

# 用我备份的 sp22 的版本
./get.sh lab00
# sp22 需要的 python 版本
uv venv --python 3.11
```

In [9]:
def summation(n, term):
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k + 1
    return total


def identity(x):
    return x


def cube(x):
    return x * x * x


def pi_term(x):
    return 8 / ((4 * x - 3) * (4 * x - 1))


def sum_naturals(n):
    return summation(n, identity)


def sum_cubes(n):
    return summation(n, cube)


def pi_sum(n):
    return summation(n, pi_term)


print(sum_naturals(100))
print(sum_cubes(100))
print(pi_sum(1000))

5050
25502500
3.141092653621038


In [10]:
def improve(update, close, guess=1):
    while not close(guess):
        guess = update(guess)
    return guess


def approx_eq(x, y, tolerance=1e-15):
    return abs(x - y) < tolerance

In [11]:
from math import sqrt


def golden_update(guess):
    return 1 / guess + 1


def square_close_to_successor(guess):
    return approx_eq(guess * guess, guess + 1)


def improve_test():
    phi = 1 / 2 + sqrt(5) / 2
    approx_phi = improve(golden_update, square_close_to_successor)
    assert approx_eq(phi, approx_phi), "phi differs from its approximation"


improve_test()

In [ ]:
def average(x, y):
    return (x + y) / 2


def nested_sqrt(a):
    def sqrt_update(x):
        return average(x, a / x)

    def sqrt_close(x):
        return approx_eq(x * x, a)

    return improve(sqrt_update, sqrt_close)


# 求平方根其实是构建一个面积为 a 的正方形求其边长
# 真实的平方根一定在 x 和 a/x 之间，为了接近真实值，求其平均值
print(nested_sqrt(256))

16.0


In [13]:
def newton_update(f, df):
    def update(x):
        return x - f(x) / df(x)

    return update


def find_zero(f, df):
    def near_zero(x):
        return approx_eq(f(x), 0)

    return improve(newton_update(f, df), near_zero)


def square_root_newton(a):
    def f(x):
        return x * x - a

    def df(x):
        return 2 * x

    return find_zero(f, df)


def power(x, n):
    product, k = 1, 0
    while k < n:
        product, k = product * x, k + 1
    return product


def nth_root_of_a(n, a):
    def f(x):
        return power(x, n) - a

    def df(x):
        return n * power(x, n - 1)

    return find_zero(f, df)


print(square_root_newton(64))
print(nth_root_of_a(2, 64))
print(nth_root_of_a(3, 64))
print(nth_root_of_a(6, 64))

8.0
8.0
4.0
2.0


In [18]:
def curried_pow(x):
    def h(y):
        return pow(x, y)

    return h


print(pow(2, 3))
print(curried_pow(2)(3))


def map_to_range(start, end, f):
    while start < end:
        print(f(start))
        start += 1


map_to_range(0, 10, curried_pow(2))


def curry2(f):
    def g(x):
        def h(y):
            return f(x, y)

        return h

    return g


def uncurry2(g):
    def f(x, y):
        return g(x)(y)

    return f


pow_curried = curry2(pow)
print(pow_curried(2)(5))
print(uncurry2(pow_curried)(2, 5))


8
8
1
2
4
8
16
32
64
128
256
512
32
32


In [ ]:
def trace(fn):
    def wrapped(x):
        print("-> ", fn, "(", x, ")")
        return fn(x)

    return wrapped


# 等价于 triple = trace(triple)
@trace
def triple(x):
    return 3 * x


triple(12)

->  <function triple at 0x71e94af64a90> ( 12 )


36